In [1]:
import pandas as pd

# 날짜 컬럼 방지
def read_csv_auto_dates(path):
    # 1) 전체 파일을 먼저 읽기
    temp = pd.read_csv(path, low_memory=False)
    
    # 2) _at 로 끝나는 컬럼 자동 탐지
    date_cols = [col for col in temp.columns if col.endswith('_at')]
    
    # 3) 다시 날짜 컬럼을 datetime 으로 읽기
    return pd.read_csv(path, parse_dates=date_cols, low_memory=False)

# Load Files (전처리)
acq = read_csv_auto_dates("./clean_data_v1/clean_acquisitions_final.csv")
deg = read_csv_auto_dates("./clean_data_v1/clean_degrees_final.csv")
frs = read_csv_auto_dates("./clean_data_v1/clean_fr_final.csv")
fds = read_csv_auto_dates("./clean_data_v1/clean_funds_final.csv")
inv = read_csv_auto_dates("./clean_data_v1/clean_investments_final.csv")
ipo = read_csv_auto_dates("./clean_data_v1/clean_ipos_final.csv")
mil = read_csv_auto_dates("./clean_data_v1/clean_milestones_final.csv")
obj = read_csv_auto_dates("./clean_data_v1/clean_objects_final.csv")
peo = read_csv_auto_dates("./clean_data_v1/clean_people_final.csv")
off = read_csv_auto_dates("./clean_data_v1/clean_offices_final.csv")
rel = read_csv_auto_dates("./clean_data_v1/clean_relationships_final.csv")

# Load Files (작업용 전처리)
company = read_csv_auto_dates("./entity_type/objects_company.csv")
finorg = read_csv_auto_dates("./entity_type/objects_finorg.csv")
person = read_csv_auto_dates("./entity_type/objects_person.csv")
product = read_csv_auto_dates("./entity_type/objects_product.csv")

# statistic join load
startup_profile = read_csv_auto_dates("./statistic_join/cy_startup_profile.csv")

print("="*60)
print("Dataset 로드 완료!")
print("="*60)

Dataset 로드 완료!


# 가설 검정 1: 로지스틱 회귀

### [스타트업의 특성] → 투자 여부

3-1. vc의 투자 의사결정은 스타트업의 특징에 따라 유의미한 영향을 미친다.

(변수: 산업, 지역, 라운드, 조달 금액 공개여부, 사무실 위치, 관계 변화량)

⭐️ 종속변수: is_vc_invested (0/1)

🌱 설명변수(X): 

- 산업 (obj_category_filled)

- 지역 (obj_city_fixed)

- 투자 단계 (num_fr_type)

- 조달 금액 공개 여부 (is_obj_funding_total_usd_private)

- 사무실 위치 (office_city)

- 관계 변화량 (relationship_growth)

In [2]:
startup_profile

,objects_cfpr_id,founded_at,closed_at,description,country_code,obj_city_fixed,first_investment_at,last_investment_at,investment_rounds,invested_companies,...,office_id,office_city,rel_cf_id,relationship_growth,funding_round_id,cat_fr_type,num_fr_type,raised_amount_usd,is_fr_raised_private,is_vc_invested
0,c:1,2005-10-17,NaT,Technology Platform Company,USA,seattle,NaT,NaT,0,0,...,1.0,Seattle,c:1,7,888.0,series-a,1,5250000.0,0,1
1,c:10,NaT,NaT,NaN,USA,culver city,NaT,NaT,0,0,...,10.0,Culver City,c:10,6,NaN,NaN,0,NaN,0,0
2,c:100,NaT,NaT,NaN,USA,san mateo,NaT,NaT,0,0,...,97.0,San Mateo,c:100,8,NaN,NaN,0,NaN,0,0
3,c:10000,2008-07-26,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0
4,c:10001,2008-07-26,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462615,r:9995,NaT,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0
462616,r:9996,NaT,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0
462617,r:9997,NaT,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0
462618,r:9998,NaT,NaT,NaN,NaN,NaN,NaT,NaT,0,0,...,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0,0


In [3]:
startup_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462620 entries, 0 to 462619
Data columns (total 31 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   objects_cfpr_id                   462620 non-null  object        
 1   founded_at                        100441 non-null  datetime64[ns]
 2   closed_at                         2809 non-null    datetime64[ns]
 3   description                       95005 non-null   object        
 4   country_code                      95043 non-null   object        
 5   obj_city_fixed                    90653 non-null   object        
 6   first_investment_at               16952 non-null   datetime64[ns]
 7   last_investment_at                16952 non-null   datetime64[ns]
 8   investment_rounds                 462620 non-null  int64         
 9   invested_companies                462620 non-null  int64         
 10  first_funding_at                

In [4]:
startup_profile['is_vc_invested'].value_counts(normalize=True)

is_vc_invested
0    0.961322
1    0.038678
Name: proportion, dtype: float64

In [5]:
# 다중공선성 및 로지스틱 회귀 사용할 컬럼
cols = [
    'is_vc_invested',
    'obj_category_filled',
    'obj_city_fixed',
    'num_fr_type',
    'is_obj_funding_total_usd_private',
    'office_city',
    'relationship_growth'  
]

df= startup_profile[cols].copy()

# 도시/사무실/산업은 상위 N개만 더미로 줄이기
def top_n_other(s, n=30, other="Other"):
    s = s.fillna("Missing") 
    top = s.value_counts().head(n).index
    return s.where(s.isin(top), other)

df2 = df.copy()
df2['obj_city_fixed'] = top_n_other(df2['obj_city_fixed'], n=15)
df2['office_city'] = top_n_other(df2['office_city'], n=10)
df2['obj_category_filled'] = top_n_other(df2['obj_category_filled'], n=10)


# 범주형 변수 인코딩
df_model = pd.get_dummies(
    df2,
    columns=[
        'obj_category_filled',
        'obj_city_fixed',
        'office_city'
    ],
    drop_first=True
)


In [6]:
# vif 다중공선성 확인

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 종속변수 제외
X = df_model.drop(columns=['is_vc_invested']).copy()

# 형변환
X = X.astype(float)

# X를 “무조건 숫자”로 강제
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

# 상수항 추가
X = sm.add_constant(X)

# VIF 계산
vif_df = pd.DataFrame()
vif_df["variable"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_df)

                            variable           VIF
0                              const  32497.787940
1                        num_fr_type      1.134344
2   is_obj_funding_total_usd_private      1.049065
3                relationship_growth      1.011382
4          obj_category_filled_Other      7.779901
5    obj_category_filled_advertising      2.017679
6     obj_category_filled_consulting      1.835309
7      obj_category_filled_ecommerce      2.798431
8     obj_category_filled_enterprise      1.748475
9    obj_category_filled_games_video      2.221603
10         obj_category_filled_media      1.780736
11        obj_category_filled_mobile      2.190894
12      obj_category_filled_software     14.335822
13           obj_category_filled_web      3.506612
14              obj_city_fixed_Other   1494.576149
15            obj_city_fixed_atlanta     18.953844
16             obj_city_fixed_austin     66.801893
17             obj_city_fixed_boston     22.635678
18          obj_city_fixed_camb

수치형 변수들: 다중공선성 없음.

산업 더미: software가 가장 많은 관측치 -> 각 산업별 계수 해석에만 유의

지역: obj_city_fixed 와 office_city가 같은 공간 정보 => 사무실 위치 제외

In [7]:
# 로지스틱 회귀용 사용할 컬럼
cols = [
    'is_vc_invested',
    'obj_category_filled',
    'obj_city_fixed',
    'num_fr_type',
    'is_obj_funding_total_usd_private',
    'relationship_growth'  
]

df= startup_profile[cols].copy()

def process_top_n(s, n=10, other="Other", missing="Missing"):
    # 1. 결측치 처리 및 상위 n개 추출
    s = s.fillna(missing)
    top_n_idx = s[s != missing].value_counts().head(n).index
    
    # 2. Top-n 및 Missing 외에는 Other로 변경
    s = s.where(s.isin(top_n_idx) | (s == missing), other)
    
    # 3. Categorical 변환 (Other를 가장 먼저 배치하여 drop_first 시 기준이 되게 함)
    # 실제 존재하는 값들 중 Other와 Missing을 제외한 나머지 정렬
    remains = sorted([x for x in s.unique() if x not in [other, missing]])
    categories = [other, missing] + remains
    
    return pd.Categorical(s, categories=categories, ordered=True)

# --- 적용 ---
df3 = startup_profile[cols].copy()

for col in ['obj_city_fixed', 'obj_category_filled']:
    df3[col] = process_top_n(df3[col], n=10)

# get_dummies 실행 (drop_first=True로 인해 각 컬럼의 첫 번째 카테고리인 'Other'가 자동 제거됨)
df_model_lo = pd.get_dummies(df3, columns=['obj_category_filled', 'obj_city_fixed'], drop_first=True)

# 전부 숫자형으로 강제 
df_model_lo = df_model_lo.astype(float)


In [8]:
[c for c in df_model_lo.columns if 'obj_city_fixed_Other' in c or 'obj_category_filled_Other' in c]

[]

In [9]:
set(df3['obj_category_filled'].unique()) 

{'Missing',
 'Other',
 'advertising',
 'biotech',
 'consulting',
 'ecommerce',
 'enterprise',
 'games_video',
 'media',
 'mobile',
 'software',
 'web'}

In [10]:
set(df2['obj_city_fixed'].unique()) 

{'Missing',
 'Other',
 'atlanta',
 'austin',
 'boston',
 'cambridge',
 'chicago',
 'london',
 'los angeles',
 'new york',
 'palo alto',
 'paris',
 'san diego',
 'san francisco',
 'seattle',
 'toronto'}

In [11]:
# 로지스틱 회귀
import statsmodels.api as sm

# x, y 분리
X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

# 전부 숫자로 강제
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = X.astype(float)

# 상수항 추가
X = sm.add_constant(X)

logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.097592
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462594
Method:                           MLE   Df Model:                           25
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4039
Time:                        21:53:56   Log-Likelihood:                -45148.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6268      0.019    -86.9

In [12]:
import numpy as np
import pandas as pd

# 오즈비 계산
odds_ratio = np.exp(result.params)

# 신뢰구간도 같이 (권장)
conf = result.conf_int()
conf.columns = ['2.5%', '97.5%']

or_df = pd.concat(
    [odds_ratio, np.exp(conf), result.pvalues],
    axis=1
)

or_df.columns = ['odds_ratio', 'or_2.5%', 'or_97.5%', 'p_value']

# 보기 좋게 정리
or_df = or_df.sort_values('odds_ratio', ascending=False)
or_df


,odds_ratio,or_2.5%,or_97.5%,p_value
obj_city_fixed_palo alto,3.107738,2.677370,3.607285,2.864589e-50
obj_city_fixed_san francisco,2.946674,2.732427,3.177721,3.104798e-173
obj_city_fixed_paris,1.942647,1.644389,2.295004,5.794565e-15
is_obj_funding_total_usd_private,1.672009,1.576331,1.773494,1.554373e-65
obj_city_fixed_seattle,1.612972,1.373192,1.894621,5.809964e-09
obj_city_fixed_austin,1.440112,1.217647,1.703221,2.043790e-05
obj_category_filled_enterprise,1.399744,1.285561,1.524070,9.507589e-15
obj_city_fixed_new york,1.383014,1.271987,1.503732,3.088768e-14
obj_category_filled_biotech,1.222311,1.130848,1.321171,4.218880e-07
obj_category_filled_mobile,1.067494,0.987954,1.153439,9.829016e-02


## 산업 x 지역 교호항 추가해서 상호작용 보기
- biotech  ↔ palo alto 

- Enterprise  ↔ san francisco 

- enterprise  ↔ Austin

- finance ↔ New York 

- consulting ↔ Paris

- mobile ↔ Seattle 

In [ ]:
# 교호항 생성
# enterprise

interaction_pairs = {
    # biotech  ↔ palo alto 
    "enterprise_palo_alto": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_palo alto"
    ),
    # Enterprise  ↔ san francisco
    "enterprise_san_francisco": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_san francisco"
    ),
    # enterprise  ↔ Austin
    "enterprise_austin": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_austin"
    ),
    # enterprise ↔ New York 
    "enterprise_new_york": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_new york"
    ),
    # enterprise ↔ Paris
    "enterprise_paris": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_paris"
    ),
    # enterprise ↔ Seattle
    "enterprise_seattle": (
        "obj_category_filled_enterprise",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [14]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097579
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462588
Method:                           MLE   Df Model:                           31
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4040
Time:                        21:53:58   Log-Likelihood:                -45142.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6256      0.019    -86.8

In [ ]:
# 교호항 생성
# biotech

interaction_pairs = {
    # biotech  ↔ palo alto 
    "biotech_palo_alto": (
        "obj_category_filled_biotech",
        "obj_city_fixed_palo alto"
    ),
    # biotech  ↔ san francisco
    "biotech_san_francisco": (
        "obj_category_filled_biotech",
        "obj_city_fixed_san francisco"
    ),
    # biotech  ↔ Austin
    "biotech_austin": (
        "obj_category_filled_biotech",
        "obj_city_fixed_austin"
    ),
    # biotech ↔ New York 
    "biotech_new_york": (
        "obj_category_filled_biotech",
        "obj_city_fixed_new york"
    ),
    # biotech ↔ Paris
    "biotech_paris": (
        "obj_category_filled_biotech",
        "obj_city_fixed_paris"
    ),
    # biotech ↔ Seattle
    "biotech_seattle": (
        "obj_category_filled_biotech",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [16]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097542
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462582
Method:                           MLE   Df Model:                           37
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4042
Time:                        21:57:20   Log-Likelihood:                -45125.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6289      0.019    -86.9

In [ ]:
# 교호항 생성
# mobile

interaction_pairs = {
    # mobile  ↔ palo alto 
    "mobile_palo_alto": (
        "obj_category_filled_mobile",
        "obj_city_fixed_palo alto"
    ),
    # mobile  ↔ san francisco
    "mobile_san_francisco": (
        "obj_category_filled_mobile",
        "obj_city_fixed_san francisco"
    ),
    # mobile  ↔ Austin
    "mobile_austin": (
        "obj_category_filled_mobile",
        "obj_city_fixed_austin"
    ),
    # mobile ↔ New York 
    "mobile_new_york": (
        "obj_category_filled_mobile",
        "obj_city_fixed_new york"
    ),
    # mobile ↔ Paris
    "mobile_paris": (
        "obj_category_filled_mobile",
        "obj_city_fixed_paris"
    ),
    # mobile ↔ Seattle
    "mobile_seattle": (
        "obj_category_filled_mobile",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [18]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097523
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462577
Method:                           MLE   Df Model:                           42
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4043
Time:                        21:59:39   Log-Likelihood:                -45116.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6330      0.019    -86.8

In [ ]:
# 교호항 생성
# web

interaction_pairs = {
    # web  ↔ palo alto 
    "web_palo_alto": (
        "obj_category_filled_web",
        "obj_city_fixed_palo alto"
    ),
    # web  ↔ san francisco
    "web_san_francisco": (
        "obj_category_filled_web",
        "obj_city_fixed_san francisco"
    ),
    # web  ↔ Austin
    "web_austin": (
        "obj_category_filled_web",
        "obj_city_fixed_austin"
    ),
    # web ↔ New York 
    "web_new_york": (
        "obj_category_filled_web",
        "obj_city_fixed_new york"
    ),
    # web ↔ Paris
    "web_paris": (
        "obj_category_filled_web",
        "obj_city_fixed_paris"
    ),
    # web ↔ Seattle
    "web_seattle": (
        "obj_category_filled_web",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [20]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097512
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462571
Method:                           MLE   Df Model:                           48
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4044
Time:                        22:10:10   Log-Likelihood:                -45111.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6340      0.019    -86.4

In [ ]:
# 교호항 생성
# software

interaction_pairs = {
    # software  ↔ palo alto 
    "software_palo_alto": (
        "obj_category_filled_software",
        "obj_city_fixed_palo alto"
    ),
    # software  ↔ san francisco
    "software_san_francisco": (
        "obj_category_filled_software",
        "obj_city_fixed_san francisco"
    ),
    # software  ↔ Austin
    "software_austin": (
        "obj_category_filled_software",
        "obj_city_fixed_austin"
    ),
    # software ↔ New York 
    "software_new_york": (
        "obj_category_filled_software",
        "obj_city_fixed_new york"
    ),
    # software ↔ Paris
    "software_paris": (
        "obj_category_filled_software",
        "obj_city_fixed_paris"
    ),
    # software ↔ Seattle
    "software_seattle": (
        "obj_category_filled_software",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [22]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097482
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462565
Method:                           MLE   Df Model:                           54
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4046
Time:                        22:12:17   Log-Likelihood:                -45097.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6277      0.019    -85.4

In [ ]:
# 교호항 생성
# ecommerce

interaction_pairs = {
    # ecommerce  ↔ palo alto 
    "ecommerce_palo_alto": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_palo alto"
    ),
    # ecommerce  ↔ san francisco
    "ecommerce_san_francisco": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_san francisco"
    ),
    # ecommerce  ↔ Austin
    "ecommerce_austin": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_austin"
    ),
    # ecommerce ↔ New York 
    "ecommerce_new_york": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_new york"
    ),
    # ecommerce ↔ Paris
    "ecommerce_paris": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_paris"
    ),
    # ecommerce ↔ Seattle
    "ecommerce_seattle": (
        "obj_category_filled_ecommerce",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [26]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097478
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462559
Method:                           MLE   Df Model:                           60
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4046
Time:                        22:15:05   Log-Likelihood:                -45095.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6265      0.019    -84.8

In [ ]:
# 교호항 생성
# games_video

interaction_pairs = {
    # games_video  ↔ palo alto 
    "games_video_palo_alto": (
        "obj_category_filled_games_video",
        "obj_city_fixed_palo alto"
    ),
    # games_video  ↔ san francisco
    "games_video_san_francisco": (
        "obj_category_filled_games_video",
        "obj_city_fixed_san francisco"
    ),
    # games_video  ↔ Austin
    "games_video_austin": (
        "obj_category_filled_games_video",
        "obj_city_fixed_austin"
    ),
    # games_video ↔ New York 
    "games_video_new_york": (
        "obj_category_filled_games_video",
        "obj_city_fixed_new york"
    ),
    # games_video ↔ Paris
    "games_video_paris": (
        "obj_category_filled_games_video",
        "obj_city_fixed_paris"
    ),
    # games_video ↔ Seattle
    "games_video_seattle": (
        "obj_category_filled_games_video",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [28]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097475
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462553
Method:                           MLE   Df Model:                           66
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4046
Time:                        22:16:40   Log-Likelihood:                -45094.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6271      0.019    -84.3

In [ ]:
# 교호항 생성
# advertising

interaction_pairs = {
    # advertising  ↔ palo alto 
    "advertising_palo_alto": (
        "obj_category_filled_advertising",
        "obj_city_fixed_palo alto"
    ),
    # advertising  ↔ san francisco
    "advertising_san_francisco": (
        "obj_category_filled_advertising",
        "obj_city_fixed_san francisco"
    ),
    # advertising  ↔ Austin
    "advertising_austin": (
        "obj_category_filled_advertising",
        "obj_city_fixed_austin"
    ),
    # advertising ↔ New York 
    "advertising_new_york": (
        "obj_category_filled_advertising",
        "obj_city_fixed_new york"
    ),
    # advertising ↔ Paris
    "advertising_paris": (
        "obj_category_filled_advertising",
        "obj_city_fixed_paris"
    ),
    # advertising ↔ Seattle
    "advertising_seattle": (
        "obj_category_filled_advertising",
        "obj_city_fixed_seattle"
    ),
}

for name, (ind_col, city_col) in interaction_pairs.items():
    if ind_col in df_model_lo.columns and city_col in df_model_lo.columns:
        df_model_lo[f"inter_{name}"] = (
            df_model_lo[ind_col] * df_model_lo[city_col]
        )


In [30]:
import statsmodels.api as sm

X = df_model_lo.drop(columns=['is_vc_invested'])
y = df_model_lo['is_vc_invested']

X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
X = sm.add_constant(X)

model_inter = sm.Logit(y, X)
result_inter = model_inter.fit()

print(result_inter.summary())


Optimization terminated successfully.
         Current function value: 0.097447
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:         is_vc_invested   No. Observations:               462620
Model:                          Logit   Df Residuals:                   462547
Method:                           MLE   Df Model:                           72
Date:                Fri, 19 Dec 2025   Pseudo R-squ.:                  0.4048
Time:                        22:18:40   Log-Likelihood:                -45081.
converged:                       True   LL-Null:                       -75739.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -1.6157      0.019    -83.0